In [1]:
#import the required files
import pandas as pd
import os 
import openpyxl

In [34]:

# function to process names and values on directory level not much calcuations involved
def process_file(directory):
    
    # defined the required variable for use
    new_filename=[]
    drawing_index=[]
    drawing_type=[]
    dp_number=[]
    
   
    
    #created 13 columns as per mentioned in the sample csv output#
    df= pd.DataFrame(columns=['Result', 'Drawing_Index', 'Drawing_Type', 'DP_Number','Drawing_Id','Weight','Dimension_Type_Nominal_Count',
                              'Dimension_Type_Nominal_Tolerance','Dimension_Type_Round_Count','Dimension_Type_Round_Tolerance','View_Count',
                              'Inclined_Drilling_Count','Inclined_Drilling_Values'])

    
    # iterate over the directory length
    for filename in os.listdir(directory):
        
        
        # call get_drawing_info function for passing the filename one by one 
        get_drawing_info(directory+"/"+filename)
        # remove the .pdf-Results from the directorey name
        filename = filename.replace(".pdf-Results","")
        new_filename.append(filename)
        
        # split filename on '_' for finding index, type & number
        file_part= filename.split('_')
        
        # condition for asigning the specific split parts to respective index
        if len(file_part)==5:
            drawing_index.append(file_part[1])
            drawing_type.append(file_part[3])
            dp_number.append(file_part[4])
        else:
            drawing_index.append("")
            drawing_type.append("")
            dp_number.append("")
    
#     just print statements for checking remove after use
#     print(new_filename)   
    
    df['Result']= new_filename # assign the Result Column to respective new value of filename
    df['Drawing_Index']= drawing_index # assign the Drawing Index Column to respective filename part
    df['Drawing_Type']= drawing_type # assign the Drawing Type Column to respective to respective filename part
    df['DP_Number']=dp_number # assign the DP Number Column to respective to respective filename part
    
#   just print statements for checking remove after use
#     print(df)

In [41]:
def get_drawing_info(directory):
    print("get drawing info function",directory)
    excel_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.xls'):
                excel_files.append(os.path.join(root, file))
    
    print("excel_files",excel_files)
    
    

In [42]:
def main():
     # directory path mentioned
    # try to ad dynamic path parsing
    directory = 'D:/My doc/ImportantDoc/Task/Challenge_and_Instructions_Workstudent_Data_Analyst/Challenge_and_Instructions_Workstudent_Data_Analyst/Results_Task_edited'
    
    process_file(directory)

    
if __name__=='__main__':
    main()

get drawing info function D:/My doc/ImportantDoc/Task/Challenge_and_Instructions_Workstudent_Data_Analyst/Challenge_and_Instructions_Workstudent_Data_Analyst/Results_Task_edited/Res0001_AC_4_ETZ_001.pdf-Results
excel_files ['D:/My doc/ImportantDoc/Task/Challenge_and_Instructions_Workstudent_Data_Analyst/Challenge_and_Instructions_Workstudent_Data_Analyst/Results_Task_edited/Res0001_AC_4_ETZ_001.pdf-Results\\Page 1\\Human Friendly Results.xls', 'D:/My doc/ImportantDoc/Task/Challenge_and_Instructions_Workstudent_Data_Analyst/Challenge_and_Instructions_Workstudent_Data_Analyst/Results_Task_edited/Res0001_AC_4_ETZ_001.pdf-Results\\Page 1\\~$Human Friendly Results.xls', 'D:/My doc/ImportantDoc/Task/Challenge_and_Instructions_Workstudent_Data_Analyst/Challenge_and_Instructions_Workstudent_Data_Analyst/Results_Task_edited/Res0001_AC_4_ETZ_001.pdf-Results\\Page 2\\Human Friendly Results.xls']
get drawing info function D:/My doc/ImportantDoc/Task/Challenge_and_Instructions_Workstudent_Data_Anal